In [168]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.datasets import load_wine

In [169]:
wine = load_wine()
data = wine.data
target = wine.target

In [170]:
len(np.unique(target))
count = [0,0,0]
for i in target:
    count[i] +=1
print(count)

[59, 71, 48]


In [171]:
wine_data = data[:130,:]
wine_target = target[:130]

In [172]:
np.unique(wine_target)

array([0, 1])

In [173]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(wine_data,wine_target,test_size=0.2,random_state=42)

In [174]:
X_train

array([[1.229e+01, 1.610e+00, 2.210e+00, ..., 9.060e-01, 1.820e+00,
        8.700e+02],
       [1.233e+01, 9.900e-01, 1.950e+00, ..., 1.060e+00, 2.310e+00,
        7.500e+02],
       [1.390e+01, 1.680e+00, 2.120e+00, ..., 9.100e-01, 3.330e+00,
        9.850e+02],
       ...,
       [1.438e+01, 1.870e+00, 2.380e+00, ..., 1.200e+00, 3.000e+00,
        1.547e+03],
       [1.269e+01, 1.530e+00, 2.260e+00, ..., 9.600e-01, 2.060e+00,
        4.950e+02],
       [1.234e+01, 2.450e+00, 2.460e+00, ..., 8.000e-01, 3.380e+00,
        4.380e+02]])

In [175]:
from sklearn.preprocessing import MinMaxScaler
ms = MinMaxScaler()
X_train = ms.fit_transform(X_train)
X_test = ms.fit_transform(X_test)


In [176]:
X_test

array([[0.79310345, 0.26666667, 0.64957265, 0.77941176, 0.4025974 ,
        0.79012346, 0.56603774, 0.11538462, 0.63111111, 0.63970588,
        0.28431373, 0.68965517, 0.60057061],
       [0.79310345, 0.26031746, 0.52136752, 0.14705882, 0.41558442,
        0.90740741, 0.75849057, 0.38461538, 0.58222222, 0.62205882,
        0.25490196, 0.99137931, 0.36875892],
       [0.81818182, 0.7015873 , 0.73504274, 0.        , 0.4025974 ,
        0.62962963, 0.66037736, 0.05769231, 0.28      , 0.47058824,
        0.26470588, 0.97413793, 0.40442225],
       [0.79937304, 0.24444444, 0.56410256, 0.57352941, 0.27272727,
        0.72839506, 0.72075472, 0.15384615, 0.40888889, 0.73529412,
        0.39215686, 0.56034483, 0.88231098],
       [0.        , 0.1968254 , 0.42735043, 0.92647059, 0.        ,
        0.48148148, 0.33584906, 0.73076923, 0.43555556, 0.        ,
        1.        , 0.55172414, 0.09201141],
       [1.        , 0.25714286, 0.51282051, 0.16176471, 0.42857143,
        0.9382716 , 0.64905

In [177]:
class Perceptron:
    def __init__(self,learning_rate=0.01,iterations=1000):
        self.learning_rate = learning_rate
        self.iterations = iterations
    
    def sigmoid(self,z):
        return 1/(1+np.exp(-z))
    
    def sigmoid_prime(self,X):
        linear_model = np.dot(X,self.weights)+self.bias
        return self.sigmoid(linear_model)*(1-self.sigmoid(linear_model))
        
    
    def predict(self,X):
        linear_model = np.dot(X,self.weights)+self.bias
        return self.sigmoid(linear_model)
    
    def fit_batch(self,X,y):
        num_samples,num_features = X.shape
        self.weights = np.zeros(num_features)
        self.bias = 0
        for i in range(self.iterations):
            y_pred = self.predict(X)
            loss = y_pred - y
            term = (loss*(self.sigmoid_prime(X))).astype(float)
            print(term.shape)
            dw = (1/num_samples)*(np.dot(term.T,X))
            db = (1/num_samples)*(np.sum(term))
            self.weights -= self.learning_rate*dw
            self.bias -= self.learning_rate*db
    
    def fit_stochastic(self,X,y):
        num_samples,num_features = X.shape
        self.weights = np.zeros(num_features)
        self.bias = 0
        for i in range(self.iterations):
            for j in range(num_samples):
                y_pred = self.predict(X[j])
                loss = y_pred - y[j]
                term = (loss*(self.sigmoid_prime(X[j]))).astype(float)
                dw = (1/num_samples)*(np.dot(term.T,X[j]))
                db = (1/num_samples)*term
                self.weights -= self.learning_rate*dw
                self.bias -= self.learning_rate*db
    
    def fit_momentum(self,X,y):
        num_samples,num_features = X.shape
        self.weights = np.zeros(num_features)
        self.bias = 0
        prev_v_w,prev_v_b,gamma = 0,0,0.9
        for i in range(self.iterations):
            for j in range(num_samples):
                y_pred = self.predict(X[j])
                loss = (y_pred - y[j])
                term = (loss*(self.sigmoid_prime(X[j]))).astype(float)
                dw = (1/num_samples)*(np.dot(term.T,X[j]))
                db = (1/num_samples)*term
                v_w = -gamma*prev_v_w + self.learning_rate*dw
                v_b = -gamma*prev_v_b +  self.learning_rate*db
                self.weights -= v_w
                self.bias -= v_b
                prev_v_w = v_w
                prev_v_b = v_b
            
    def test(self,X_test,y_test):
        y_predtest = self.predict(X_test)
        correct = 0
        for i in range(len(y_predtest)):
            if y_predtest[i] > 0.5:
                y_predtest[i] = 1
            else:
                y_predtest[i] = 0
            if y_predtest[i] == y_test[i]:
                correct+=1
        print(y_predtest)
        print("Accuracy is : ",correct/len(y_predtest))

In [178]:
log_reg = Perceptron()
log_reg.fit_momentum(X_train,y_train)

In [179]:
log_reg.test(X_test,y_test)

[0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0.
 1. 0.]
Accuracy is :  0.9230769230769231


In [180]:
print(y_test)

[0 0 0 0 1 0 1 1 1 0 1 0 1 0 1 1 0 1 1 1 0 0 1 0 1 0]
